In [6]:
from utils import db
from utils import s3

In [16]:
# 1. Create a a file:
# col:
# product_name varchar(20),
# manufacturingcost decimal(5,2)
# For each product build record using python script.
# 'iphon11','iphone12','iphone13','iphoneSE','IpadMax','IpadMini','laptop256','Macbook512'],
# 'Samsung':['galaxy10','galaxy11','galaxy12','galaxy13','watch320','watch340'

import random

product_costs = {}

product_names = ['iphon11','iphone12','iphone13','iphoneSE','IpadMax','IpadMini','laptop256','Macbook512', 
    'galaxy10','galaxy11','galaxy12','galaxy13','watch320','watch340', 'Nk320','Nk400','Nk500']

f = open('data/cards_ingest/product_cost.csv', 'w')
f.write('product_name' + ',' + 'manufacturing_cost\n')
for p in product_names:
    f.write(p + ',' + str(round(random.uniform(100.00, 999.99),2)) + '\n')
f.close()



In [7]:
# 2. Load data into s3 in separate folder (order data must be created earlier in class project 3).
'''
done in AWS CLI
'''


sql/create/ct_order.sql done.
sql/create/ct_product_cost.sql done.


'\ndone in AWS CLI\n'

In [13]:
# 3. Use copy command  to load data into redshift by.
db.do_query_file('sql/create/ct_order.sql')
db.do_query_file('sql/create/ct_product_cost.sql')
# db.do_query_file('sql/s3/copy_order.sql')

sql/create/ct_order.sql done.
sql/create/ct_product_cost.sql done.


In [9]:
# 4  create a view by joining order and product_cost.
query = """ 
    CREATE SCHEMA IF NOT EXISTS vw; 
"""
db.do_query(query, [])

query = """ 
CREATE OR REPLACE VIEW vw.order_product_cost AS 
    SELECT o.*, c.manufacturing_cost FROM cards_ingest.order o
    JOIN cards_ingest.product_cost c
    ON o.product_name = c.product_name;
"""
db.do_query(query, [])

In [10]:
# 5. Use boto3 to create bucket then list the buckets
s3.client.create_bucket(Bucket = 'views-quintrix-spearscjs')

print(s3.client.list_buckets(), '\n')

if(s3.bucket_exists('views-quintrix-spearscjs')):
    print("SUCCESS")
else:
    print("FAILURE")


{'ResponseMetadata': {'RequestId': 'EBKJ44T22TPSEC69', 'HostId': 'qShJnl9bVc7Bw+H0HWCBDKP9GQJ48Xui5LHW83qVnDOFhwJLyCFutbu7x67B3jPab9J5xB9rAHQ=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'qShJnl9bVc7Bw+H0HWCBDKP9GQJ48Xui5LHW83qVnDOFhwJLyCFutbu7x67B3jPab9J5xB9rAHQ=', 'x-amz-request-id': 'EBKJ44T22TPSEC69', 'date': 'Thu, 12 Jan 2023 21:48:20 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'quintrix-spearscjs', 'CreationDate': datetime.datetime(2023, 1, 9, 19, 37, 25, tzinfo=tzutc())}, {'Name': 'test-spearscjs', 'CreationDate': datetime.datetime(2023, 1, 12, 21, 39, 7, tzinfo=tzutc())}, {'Name': 'views-quintrix-spearscjs', 'CreationDate': datetime.datetime(2023, 1, 12, 19, 14, 27, tzinfo=tzutc())}], 'Owner': {'DisplayName': 'spearscjs', 'ID': '1b7f5da4c80f4cc0884c213769d24009e307959bece34048a82e0db6e42d1d19'}} 

SUCCESS


In [14]:
# 6. Unload data from view into s3 bucket (this has to be separate bucket then created earlier) (not doing yet)

# csv, header, allow overwrite, optinal? (null as)

s3.empty_bucket('views-quintrix-spearscjs')

query = """
    UNLOAD ('select * from vw.order_product_cost') 
    TO 's3://views-quintrix-spearscjs/vw_order_product_cost.csv'
    IAM_ROLE 'arn:aws:iam::775856445594:role/service-role/AmazonRedshift-CommandsAccessRole-20230109T135507'
    PARALLEL OFF
    ALLOWOVERWRITE 
    FORMAT CSV;
"""
res = db.do_query(query, [])